In [1]:
#import libaries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

In [2]:
# read the raw csv file(/s)
movie_file = ("Resources/movies_metadata.csv")

In [3]:
#read the movie file and create a dataframe

movie_data_df = pd.read_csv(movie_file)

#list the column types
movie_data_df.dtypes


/Users/saraheshelden/miniconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [4]:
#cleanup and filter steps ( budget, revenue , release year)

movie_data_df = movie_data_df.drop(columns=['adult', 'production_countries','belongs_to_collection','homepage','id','imdb_id','overview','poster_path','runtime','spoken_languages','tagline','original_title','video'])

print( f'Inital count of data from file: {len(movie_data_df)}')

clean_movie_data_df = movie_data_df.loc[movie_data_df["budget"] != 0  ]
print( f'count after filtering out $0 budget: {len(clean_movie_data_df)}')

clean_movie_data_df = clean_movie_data_df.loc[clean_movie_data_df["revenue"] != 0  ]
print( f'count after filtering out $0 revenue: {len(clean_movie_data_df)}')

clean_movie_data_df['release_year']= pd.to_datetime(clean_movie_data_df["release_date"]).dt.year 
clean_movie_data_df = clean_movie_data_df.loc[clean_movie_data_df["release_year"] < 2019 ]
print( f'count after filtering out movies in future date: {len(clean_movie_data_df)}')
git
clean_movie_data_df['net_loss_profit'] = clean_movie_data_df["revenue"] - clean_movie_data_df["budget"]



Inital count of data from file: 45466
count after filtering out $0 budget: 45466
count after filtering out $0 revenue: 7414


ValueError: Given date string not likely a datetime.

In [8]:
# extract the Genre from the column that has a list of dictionaries
clean_movie_data_df['Primary Genre']=[json.loads(row.genres.replace("'",'"'))[0]['name'] if row.genres != '[]' else '' for _,row in clean_movie_data_df.iterrows()]
print(clean_movie_data_df[['net_loss_profit','Primary Genre', 'release_year', 'title']])



KeyError: "['net_loss_profit' 'release_year'] not in index"

In [9]:
print(clean_movie_data_df.dtypes)

# convert number columns: revenue, budget, net_loss_profit to numeric for sorting purposes
clean_movie_data_df['revenue'] = pd.to_numeric(clean_movie_data_df['revenue'])
clean_movie_data_df['budget'] = pd.to_numeric(clean_movie_data_df['budget'])
clean_movie_data_df['net_loss_profit'] = pd.to_numeric(clean_movie_data_df['net_loss_profit'])
print(clean_movie_data_df.dtypes )
# sorting movie by revenue from highest to lowest
# sort_by_revenue = clean_movie_data_df.sort_values(by='revenue', ascending = False)
sort_by_netProfit = clean_movie_data_df.sort_values(by='net_loss_profit', ascending = False)

# output csv file sort by revenue data
sort_by_revenue.to_csv("revenue.csv", encoding='utf-8')

# print to terminal
sort_by_revenue.head(10)

budget                   object
genres                   object
original_language        object
popularity               object
production_companies     object
release_date             object
revenue                 float64
status                   object
title                    object
vote_average            float64
vote_count              float64
Primary Genre            object
dtype: object


ValueError: Unable to parse string "/ff9qCepilowshEtG2GYWwzt2bs4.jpg" at position 5457

In [10]:
top_10_profitable_mv = sort_by_revenue[:10][['title','net_loss_profit','Primary Genre']].reset_index()

print(top_10_profitable_mv)

# bar graph

NameError: name 'sort_by_revenue' is not defined

In [7]:
ax = top_10_profitable_mv.plot.bar(x='title', y='net_loss_profit', rot = 90)

x_axis = np.arange(len(top_10_profitable_mv))

tick_locations = [value for value in x_axis]

ax.set_title('Top 10 Most Movie Revenue up to 2018')
ax.set_xlabel('Title')
ax.set_ylabel('Net loss profit')

# ax.text(set_xlabel,set_ylabel, "String", fontsize=12)

for i in range(top_10_profitable_mv):
    plt.text(x = top_10_profitable_mv[i]-0.5 , y = bars4[i]+0.1, s = label[i], size = 6)


# plt.xticks(tick_locations, top_10_profitable_mv["net_loss_profit"], rotation = "vertical")

plt.show()
plt.tight_layout()


# ax.text(rect.get_x() + rect.get_width() / 2, height + 5, label,
#             ha='center', va='bottom')

# x_labels = [108300.0, 110540.0, 112780.0, 115020.0, 117260.0, 119500.0, 
#             121740.0, 123980.0, 126220.0, 128460.0, 130700.0]
# # ax.set_xticklabels(x_labels)
# ax.text(x_labels)



NameError: name 'top_10_profitable_mv' is not defined